In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchsummary import summary
from lion_pytorch import Lion
from torchmetrics.classification import BinaryMatthewsCorrCoef
import os
import pandas as pd
import numpy as np
import math
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader, ConcatDataset
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
learning_rate = 1e-3
mini_batch = 64
batch_size = 64
accumulation_steps = batch_size // mini_batch

num_epochs = 100
df = pd.read_csv('train_v3.csv')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

In [ ]:
class LCTDataset(Dataset):
    def __init__(self, data):
        self.data = data
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self,index):
        data1 = self.data.iloc[index, 3:327].values
        data1 = torch.from_numpy(data1).float()
        
        data2 = self.data.iloc[index, 327:].values
        data2 = torch.from_numpy(data2).float()
        
        target = self.data.iloc[index, 0]
        return data1, data2, target

In [ ]:
%%time
train_data = LCTDataset(data=df)
train_loader = torch.utils.data.DataLoader(train_data, batch_size=mini_batch, shuffle=True, num_workers=4, pin_memory=True)
print(f'Number of batches: {len(train_loader)}')

In [ ]:
class SiameseNet(nn.Module):
    def __init__(self):
        super(SiameseNet, self).__init__()
        self.body = nn.Sequential(
            nn.Linear(324, 512),
            nn.GELU(),
            nn.Linear(512, 512),
        )
        self.classifier = nn.Sequential(
            nn.SiLU(),
            nn.Linear(512, 512),
            nn.ELU(),
            nn.Linear(512, 1),
            nn.Sigmoid()
        )
        
    def forward(self, input1, input2):
        output1 = self.body(input1)
        output2 = self.body(input2)
        output = torch.abs(torch.subtract(output1, output2))
        output = self.classifier(output)
        return outputclass SiameseNet(nn.Module):
    def __init__(self):
        super(SiameseNet, self).__init__()
        self.body = nn.Sequential(
            nn.Linear(324, 512),
            nn.GELU(),
            nn.Linear(512, 512),
        )
        self.classifier = nn.Sequential(
            nn.SiLU(),
            nn.Linear(512, 512),
            nn.ELU(),
            nn.Linear(512, 1),
            nn.Sigmoid()
        )
        
    def forward(self, input1, input2):
        output1 = self.body(input1)
        output2 = self.body(input2)
        output = torch.abs(torch.subtract(output1, output2))
        output = self.classifier(output)
        return output
    
model = SiameseNet();
model.load_state_dict(torch.load('model_v5_39.pth'))
    
model = SiameseNet();
model.load_state_dict(torch.load('model_v5_39.pth'))

In [ ]:
model.to(device);

print(summary(model, [(1, 324), (1, 324)]))

In [ ]:
weights = df.copy()
weights['count_of_targets'] = weights['target'].map(weights['target'].value_counts())
weights.sort_values('target', inplace=True)
weights.drop_duplicates('target', inplace=True)
n_samples = list(weights['count_of_targets'])
weights = [(max(n_samples)/n) for n in n_samples]
loss_weights = torch.FloatTensor(weights).to(device)
loss_weights

In [ ]:
class W_BCELoss(torch.nn.Module):
    
    def __init__(self, w_n = 3.2452, w_p = 1.0000):
        super(W_BCELoss, self).__init__()
        
        self.w_p = w_p
        self.w_n = w_n
        
    def forward(self, logits, labels, epsilon = 1e-7):
        
        loss_pos = -1 * torch.mean(self.w_p * labels * torch.log(logits + epsilon))
        loss_neg = -1 * torch.mean(self.w_n * (1-labels) * torch.log((1-logits) + epsilon))
        
        loss = loss_pos + loss_neg
        
        return loss

In [ ]:
def train_fn(model, optimizer, scheduler, loss_fn, dataloader, device, epoch, accumulation_steps):
    flag = True
    model.train() 
    final_loss = 0  
    train_acc = 0
    total=0
    metric = BinaryMatthewsCorrCoef(threshold=0.75).to(device)
    loop = tqdm(enumerate(dataloader), total=len(dataloader), leave=False)
    for batch_idx, (features1, features2, labels) in loop:
        inputs1, inputs2, targets = features1.to(device, non_blocking=True), features2.to(device, non_blocking=True), labels.to(device, non_blocking=True).float()
        outputs = model(inputs1, inputs2)
        outputs = outputs.reshape(len(outputs))
        metric.update(outputs, targets)
        loss = loss_fn(outputs, targets) 
        loss.backward() 
        if (batch_idx + 1) % accumulation_steps == 0:
            optimizer.step()  
            scheduler.step() 
            optimizer.zero_grad() 
#        total += len(targets)
        final_loss += loss.item() 
#        predicted = torch.where(outputs >= 0.5, 1, 0)
#        train_acc += ((predicted == targets).sum().item())
        if (epoch + 1) % 10 == 0 and flag:
            name = 'model_v9_' + str(epoch + 1) + '.pth'
            torch.save(model.state_dict(), name)
            flag = False
        loop.set_description(f'Epoch: [{epoch+1}/{num_epochs}]') 
        loop.set_postfix(loss=final_loss/(batch_idx+1), mat=metric.compute().item())#, acc=train_acc/total*100)
    final_loss /= len(dataloader) 
#    train_acc = (train_acc/total)*100
    
    return final_loss

In [ ]:
def run_training(train_loss_lst, num_epochs, accumulation_steps):
    #using adam optimizer for optimization
    optimizer = optim.AdamW(filter(lambda x: x.requires_grad, model.parameters()), lr=learning_rate)
    scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer=optimizer, T_max = num_epochs, eta_min = 3e-4)
    loss_fn = W_BCELoss()
    
    for epoch in range(num_epochs):
        train_loss = train_fn(model, optimizer, scheduler, loss_fn, train_loader, device, epoch, accumulation_steps) #training loss and accuracy
        train_loss_lst.append(train_loss)
    
    print(f'Train finished!')
    print(f'Your last train accuracy is {train_acc_lst[-1]}%')

In [ ]:
# Fit our model
train_loss_lst = []
run_training(train_loss_lst, num_epochs, accumulation_steps)